In [1]:
import pandas as pd
import matplotlib as plt
import datetime
import typing
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots

%load_ext autoreload
%autoreload 2

In [2]:
data = pd.read_csv('../DATA/processed/dataset.csv', parse_dates=['date'])
data = data.set_index('date')

In [3]:
wind_dir = {'Ветер, дующий с востока': 0,
            'Ветер, дующий с востоко-северо-востока': 45/2,
            'Ветер, дующий с северо-востока': 45,
            'Ветер, дующий с северо-северо-востока': 45 + 45/2,
            'Ветер, дующий с севера': 90,
            'Ветер, дующий с северо-северо-запад': 90 + 45/2,
            'Ветер, дующий с северо-запада': 135,
            'Ветер, дующий с западо-северо-запада': 135 + 45/2,
            'Ветер, дующий с запада': 180,
            'Ветер, дующий с западо-юго-запада': 180+45/2,
            'Ветер, дующий с юго-запада': 225,
            'Ветер, дующий с юго-юго-запада': 225 + 45/2,
            'Ветер, дующий с юга': 270,
            'Ветер, дующий с юго-юго-востока': 270 + 45/2,
            'Ветер, дующий с юго-востока': 315,
            'Ветер, дующий с востоко-юго-востока': 315 + 45/2,
            'Штиль, безветрие': None,
            }

In [4]:
def prepare_features(data: pd.DataFrame) -> pd.DataFrame:
    """Features preparation for anomaly detection and clustering"""
    # textual values encoding
    data['wind_direction'] = data.wind_direction.fillna(method='bfill')
    data['wind_direction'] = data.wind_direction.map(wind_dir)

    data['prec_amount'] = data.prec_amount.fillna(method='bfill')
    data.loc[data.prec_amount == 'Осадков нет', 'prec_amount'] = 0
    data.loc[data.prec_amount == 'Следы осадков', 'prec_amount'] = 0
    data['prec_amount'] = data.prec_amount.astype(float)

    data['dew_point_diff'] = data.temperature - data.dew_point_temp

    # fill missing value before PCA
    data['P1'] = data.P1.interpolate()
    data['P2'] = data.P2.interpolate()
    data['pressure'] = data.pressure.interpolate()
    data['temperature'] = data.temperature.interpolate()
    data['humidity'] = data.humidity.interpolate()
    return data

In [5]:
sel_columns = ['P1','hum_meteo', 'temp_meteo', 'pres_meteo', 'wind_speed', 'prec_amount']

In [6]:
data = prepare_features(data)
data = data[sel_columns]

# Add lag variable

In [7]:
data.columns

Index(['P1', 'hum_meteo', 'temp_meteo', 'pres_meteo', 'wind_speed',
       'prec_amount'],
      dtype='object')

In [8]:
columns = ['P1',
          # 'temp_meteo', 'pres_meteo', 'hum_meteo', 'wind_direction',
          # 'wind_speed', 'prec_amount',
          ]

In [9]:
shift_values = [1, 12, 12*2, 12*4, 12*8, 12*24]

In [10]:
for c in columns:
    for s in shift_values:
        data[f'{c}_shift_{s}'] = data[c].shift(s)

# Train test split

In [11]:
sel_columns = ['hum_meteo', 'temp_meteo', 'pres_meteo', 'wind_speed', 'prec_amount']

In [12]:
y = data.P1

In [13]:
idx_split = str(data.index[-1]-datetime.timedelta(days=4))
idx_split


'2020-01-24 00:00:00'

In [14]:
X_test = data[idx_split:]
X_train = data[:idx_split]
y_test = data[idx_split:]
y_train = data[:idx_split]

In [15]:
X_train = X_train.drop(columns='P1')

In [17]:
def clear_test(X_test):
    start_idx = X_test.index[0] + datetime.timedelta(minutes=5*int(np.max(shift_values)))
    columns = ['P1']
    X_test['P1'][str(start_idx):]
    for c in columns:
        for s in shift_values:
            X_test[f'{c}_shift_{s}'][str(start_idx):] = None
    return X_test

In [18]:
def predict(X_test, model):
    start_idx = X_test.index[0] + datetime.timedelta(minutes=5*np.max(shift_values))
    for index, row in X_test[str(start_idx):].iterrows():
        columns = ['P1']
        for c in columns:
            for s in shift_values:
                row[f'{c}_shift_{s}'] = row[c][[str(index-datetime.timedelta(minutes=5*shift_values))]]
        row['P1'] = model.predict(row)
    prediction = X_test.P1[str(start_idx):]
    return prediction
        
    
    
    

In [19]:
X_test = clear_test(X_test)

//home/anatoly/HDD/anaconda3/envs/mlforair/lib/python3.7/site-packages/pandas/core/series.py:1240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_with(key, value)
//home/anatoly/HDD/anaconda3/envs/mlforair/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
model = LinearRegression()
model.fit(X_train, y_train)
prediction = predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').